In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Data

In [ ]:
from exp import *

In [ ]:
mpl.rcParams['image.cmap'] = 'gray'

In [ ]:
path = datasets.download_data(MNIST_URL, ext='.gz')

In [ ]:
with gzip.open(path, 'rb') as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

In [ ]:
x_train,y_train,x_valid,y_valid = map(tensor, (x_train,y_train,x_valid,y_valid))

In [ ]:
n,m = x_train.shape
c = y_train.max()+1

In [ ]:
n,m,c

(50000, 784, tensor(10))

## Fully connected model

In [ ]:
# num hidden
nh = 50

In [ ]:
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [ ]:
def relu(x): return x.clamp_min(0.)

In [ ]:
def model(xb):
    l1 = xb @ w1 + b1
    l2 = relu(l1)
    return (l2 @ w2 + b2).squeeze()

In [ ]:
model(x_valid).shape

torch.Size([10000])

In [ ]:
%timeit -n 10 _=model(x_valid)

3.02 ms ± 218 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Loss function: MSE

In [ ]:
def mse(output, targ): return (output-targ).pow(2).mean()

In [ ]:
y_train,y_valid = y_train.float(),y_valid.float()

In [ ]:
x_train.mean(),x_train.std()

(tensor(0.1304), tensor(0.3073))

In [ ]:
train_mean,train_std = x_train.mean(),x_train.std()

In [ ]:
def normalize(x, m, s): return (x-m)/s

In [ ]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [ ]:
train_mean,train_std = x_train.mean(),x_train.std()

In [ ]:
x_train.mean(),x_train.std()

(tensor(0.0001), tensor(1.))

In [ ]:
preds = model(x_train)

In [ ]:
preds.shape

torch.Size([50000])

In [ ]:
mse(preds, y_train)

tensor(26.7578)

## Gradients